In [ ]:
# Load in required libraries

import pandas as pd # loads pandas
import numpy as np # loads numpy
import matplotlib.pyplot as plt
import seaborn as sns # loads the seaborn library
# sets the option we want our plot output to appear inline with our results
%matplotlib inline


# What is pandas?
* Pandas is a library or framework for python created to store, manipulate, and perform calculations on data **in memory.**
* This means that it is usually faster than SQL, but comes obviously at a tradeoff for very large datasets.  That being said, it is still quite efficient
* Because SQL and pandas both can be used for tasks of manipulating data, I will try to include the SQL code for each pandas statement so that you can understand what it is doing.

# Loading in data

In [ ]:
# Open table and read it into a pandas dataframe and then save it as a variable called data
data = pd.read_csv('data/train_u6lujuX_CVtuZ9i.csv')

# SELECT top 10 * FROM data
data.head(10)

# First Steps

 - Now **data** contains a Dataframe (TABLE) that has our data
 - In *pandas* each Dataframe variable has certain *methods*, or functions attached to the Dataframe class, you can call that do different things.  The above set of code `data.head(10)` calls the method `head` and asks it to return the top 10 rows of the Dataframe
 - To learn more about any class object or method, you can just add a `?` behind it in a code cell and run it to bring up the help on it.  Try running that below for the `head` method.

In [ ]:
data.head?

- You can also type in a variable once it's been loaded in, and once you add a `.` to the end, and press the TAB key you will see an autocomplete list of the available methods. Try that below with data...

In [ ]:
data

 - You may notice also that you can access a column of the data by adding the column name behind a `.` as well.  This works for any column without a SPACE in the name.  You can always access a column with the syntax `data['LoanAmount']`.  When you select a column, you get a pandas series.  This series shares the same index as the Dataframe `data`.

In [ ]:
# SELECT LoanAmount FROM data

data.LoanAmount

In [ ]:
print(type(data))
print(type(data.LoanAmount))
print(type(data['LoanAmount']))

 - You can select multiple columns by putting them in a list.  This means that you add a set of [] around a comma separated list of column names.  

In [ ]:
# SELECT TOP 5 Credit_History, LoanAmount FROM data

colsToSelect = ['Credit_History','LoanAmount']

data[colsToSelect].head()
# This is often just expressed like data[['Credit_History','LoanAmount']], but note the 2 sets of brackets

 - What sort of values are in Credit_History?  We can run the method `value_counts` to see a list with the number of records that have each value.  This list is sorted in descending order

In [ ]:
data['Credit_History'].value_counts()

 - In python, you can chain multiple objects and methods together with the `.`  This can lead to very efficient code where you perform several actions and then do something with the result.  In the section below, we will grab a section of the data, manipulate it, and then plot the results all in one line of code, but here we can keep it broken into pieces

In [ ]:
propTypeCounts = data['Property_Area'].value_counts()
propTypeCounts.plot(kind='bar')

### Using Group By to look into relationships

Let's start with a simple groupby to learn more about the relationship between Loan_Status and Credit_History

In [ ]:
# SELECT Credit_History, Loan_Status, count(*) FROM data groupby Credit_History, Loan_Status

data.groupby(['Credit_History', 'Loan_Status']).size()

 - In pandas, we can take this one step further and formulate the results quickly into a 'Crosstab' query to get a format that can be easily graphed by calling the unstack() function.

In [ ]:
data.groupby(['Credit_History', 'Loan_Status']).size().unstack()

In [ ]:
data.groupby(['Credit_History', 'Loan_Status']).size().unstack().plot(kind='bar')

### Let's look at income and loan amount
 - The info method will list each of the columns in the DataFrame.

In [ ]:
data.info()

 - We can see that there is a column for **ApplicantIncome** and **CoapplicantIncome**.  Let's create a new column **TotalIncome**

In [ ]:
data['TotalIncome'] = data['ApplicantIncome'] + data['CoapplicantIncome']
data.head()

#### Let's try and showoff some cool graphs
 - These may have some sections that look kind of advanced, but it's just to show how the graphs can be customized and labeled to be used in presentations.

In [ ]:
# We dont Probably need the Gridlines. Do we? If yes comment this line
sns.set(style="ticks")

# Here we create a matplotlib axes object. The extra parameters we use 
# "ci" to remove confidence interval
# "marker" to have a x as marker. 
# "scatter_kws" to provide style info for the points.[s for size]
# "line_kws" to provide style info for the line.[lw for line width]

g = sns.regplot(x='LoanAmount', y='TotalIncome', data=data, ci = False, 
    scatter_kws={"color":"darkred","alpha":0.3,"s":90},
    line_kws={"color":"g","alpha":0.5,"lw":4},
    marker="x")

# remove the top and right line in graph
sns.despine()

# Set the size of the graph from here
g.figure.set_size_inches(8,5)
# Set the Title of the graph from here
g.axes.set_title('Total Income vs. Loan Amount', fontsize=34,color="r",alpha=0.5)
# Set the xlabel of the graph from here
g.set_xlabel("Loan Amount",size = 32,color="r",alpha=0.5)
# Set the ylabel of the graph from here
g.set_ylabel("Total Income",size = 32,color="r",alpha=0.5)
# Set the ticklabel size and color of the graph from here
g.tick_params(labelsize=14,labelcolor="black")

#### Let's do one more to see if this relationship changes for loans that were either rejected or approved

In [ ]:
# So this function creates a faceted plot. The plot is parameterized by the following:

# col : divides the data points into days and creates that many plots
# palette: deep, muted, pastel, bright, dark, and colorblind. change the colors in graph. Experiment with these
# col_wrap: we want 2 graphs in a row? Yes.We do
# scatter_kws: attributes for points
# hue: Colors on a particular column.
# size: controls the size of graph

g = sns.lmplot(x='LoanAmount', y='TotalIncome',ci=None,data=data, col='Loan_Status',
    palette="muted",col_wrap=2,scatter_kws={"s": 100,"alpha":.5},
    line_kws={"lw":4,"alpha":0.5},hue='Loan_Status',x_jitter=1.0,y_jitter=1.0,size=6)

# remove the top and right line in graph
sns.despine()
# Additional line to adjust some appearance issue
plt.subplots_adjust(top=0.85)

# Set the Title of the graph from here
g.fig.suptitle('Total Income vs Loan Amount', fontsize=34,color="r",alpha=0.5)

# Set the xlabel of the graph from here
g.set_xlabels("Loan Amount",size = 32,color="r",alpha=0.5)

# Set the ylabel of the graph from here
g.set_ylabels("Total Income",size = 32,color="r",alpha=0.5)

# Set the ticklabel size and color of the graph from here
titles = ['Approved','Rejected']
for ax,title in zip(g.axes.flat,titles):
    ax.tick_params(labelsize=14,labelcolor="black")
    ax.set_title(title)